In [1]:
import Pkg; Pkg.add("Combinatorics")

    Updating registry at `C:\Users\Personal\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Personal\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Personal\.julia\environments\v1.7\Manifest.toml`


In [2]:
#!/bin/julia
using JuMP, Gurobi, Combinatorics, DelimitedFiles, LinearAlgebra;

In [3]:
function nodos(W)
    #============================================================
    Descripcion de la funcion:
    
    ============================================================#
    nodos = []
    cantidad_de_aristas = size(W)[1] 
    for i in 1:cantidad_de_aristas
        for j in 1:2
            if W[i,j] in nodos
                continue 
            else 
                append!(nodos, [W[i,j]])
            end
        end
    end
    output = nodos,length(nodos)
return output
end

function matriz_de_adyacencia(W)
    #=======================================================================
    Descripcion de la funcion:
    Recibe un grafo W y retorna su matriz de adyacencia, que resulta ser una
    matriz cuadrada simetrica.
    =======================================================================#
    # Arreglos con los elementos del grafo
    vertices = nodos(W)[1]
    aristas = W
    # cardinales en el grafo
    cantidad_de_vertices = nodos(W)[2]
    cantidad_de_aristas = size(W)[1]
    # Matriz de adyacencia
    matriz_de_adyacencia = zeros(cantidad_de_vertices, cantidad_de_vertices)
    for e in 1:cantidad_de_aristas
        #===================================================================
        Matriz de adyacencia:
        En el termino (i,j) guarda el peso de la arista que conecta los
        nodos i,j, en caso que no haya arista el peso sera zero
        ===================================================================#
        matriz_de_adyacencia[W[e,1], W[e,2]] = W[e,3];
        matriz_de_adyacencia[W[e,2], W[e,1]] = W[e,3];
    end
    output = matriz_de_adyacencia
return output
end

matriz_de_adyacencia (generic function with 1 method)

In [4]:
# Test de la funcion nodos
println("Los vertices de la instancia 1 son:")
@show nodos(W1)
println("Los vertices de la instancia 2 son:")
@show nodos(W2)

Los vertices de la instancia 1 son:


LoadError: UndefVarError: W1 not defined

In [5]:
# Test de la funcion matriz_de_adyacencia
println("La matriz de adyacencia de la instancia 1 es:")
@show matriz_de_adyacencia(W1)

La matriz de adyacencia de la instancia 1 es:


LoadError: UndefVarError: W1 not defined

In [6]:
# Test de la funcion matriz_de_adyacencia
println("La matriz de adyacencia de la instancia 2 es:")
@show matriz_de_adyacencia(W2)

La matriz de adyacencia de la instancia 2 es:


LoadError: UndefVarError: W2 not defined

In [7]:
function funMST2(W)
    mdl = Model()

    #Le indicamos a JuMP que el solver a utilizar es Gurobi y eliminamos presolver
    set_optimizer(mdl, Gurobi.Optimizer)
    set_optimizer_attributes(mdl, "Presolve" => 0, "OutputFlag" => 0) 
    
    # Arreglos con los elementos del grafo
    vertices = nodos(W)[1]
    aristas = W
    # cardinales en el grafo
    cantidad_de_vertices = nodos(W)[2]
    cantidad_de_aristas = size(W)[1]
    # Matriz de adyacencia
    matriz = matriz_de_adyacencia(W)
    
    #=======================================================================
    Variables del problema
    =======================================================================#
    x = @variable(mdl, x[i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices], Bin)
    #=======================================================================
    Restricciones del problema
    =======================================================================#
    # Condicion de simetria de la matriz
    for i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices
        if matriz[i,j] != 0
            @constraint(mdl, x[i,j] == x[j,i])
        end
    end
    # Condicion para no considerar aristas imaginarias de peso zero
    for i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices
        if matriz[i,j] == 0
            @constraint(mdl, x[i,j] == 0)
        end
    end
    # Condicion de arbol
    @constraint(mdl, sum(x)/2 == cantidad_de_vertices -1)
    # Condicion de corte
    for S in powerset(1:cantidad_de_vertices)
        if (length(S)>0) && (length(S) < cantidad_de_vertices)
            @constraint(mdl, sum(x[i,j] for i in setdiff(1:cantidad_de_vertices,S), j in S) >= 1)
        end
    end
    #=======================================================================
    Funcion objetivo y optimizacion
    =======================================================================#
    @objective(mdl, Min, sum(x[i,j] * matriz[i,j] / 2 for i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices))
    optimize!(mdl)
    return(objective_value(mdl));
end

funMST2 (generic function with 1 method)

In [10]:
# Test de la funcion funMST2 para la instancia 1
W1=readdlm("instancia1.txt", ',', Int, '\n')
funMST2(W1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-09
Set parameter Presolve to value 0
[1 2 1; 2 3 1; 3 4 3; 4 5 1; 5 6 1; 6 7 1; 7 1 2; 3 5 1]

In [9]:
# Test de la funcion funMST2 para la instancia 2
W2=readdlm("instancia2.txt", ',', Int, '\n')
funMST2(W2)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-09
Set parameter Presolve to value 0


38.0